In [1]:
import pandas as pd
import requests
import json
import base64
import tensorflow as tf

In [2]:
data = pd.read_csv('Campus_Selection_Clean.csv')
inputs = data.iloc[27, :12].to_dict()
inputs

{'gender': 'M',
 'ssc_p': 63.0,
 'ssc_b': 'Others',
 'hsc_p': 67.0,
 'hsc_b': 'Others',
 'hsc_s': 'Commerce',
 'degree_p': 66.0,
 'degree_t': 'Comm&Mgmt',
 'workex': 'No',
 'etest_p': 68.0,
 'specialisation': 'Mkt&HR',
 'mba_p': 57.69}

In [3]:
data.iloc[27, :13]

gender                    M
ssc_p                  63.0
ssc_b                Others
hsc_p                  67.0
hsc_b                Others
hsc_s              Commerce
degree_p               66.0
degree_t          Comm&Mgmt
workex                   No
etest_p                68.0
specialisation       Mkt&HR
mba_p                 57.69
status                    1
Name: 27, dtype: object

In [4]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [5]:
json_data = prepare_json(inputs)

endpoint = "https://campus-selection-classification-production.up.railway.app/v1/models/campus-selection-classification-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Not Accepted" if prediction_value < 0.5 else "Accepted"
else:
    result = "Error: No predictions found in response."

In [6]:
print(result)

Accepted
